## Load necessary modules

In [7]:
# show images inline
%matplotlib inline

# automatically reload modules when they have changed
%load_ext autoreload
%autoreload 2

# import keras
import keras
import pandas as pd
# import keras_retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color
from keras_retinanet.utils.gpu import setup_gpu


# import miscellaneous modules
from IPython.display import clear_output
import matplotlib
import matplotlib.pyplot as plt
import cv2
import os
import glob
import numpy as np
import time

# use this to change which GPU to use
gpu = "0"

# set the modified tf session as backend in keras
setup_gpu(gpu)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
1 Physical GPUs, 1 Logical GPUs


## Load RetinaNet model

In [8]:
# adjust this to point to your downloaded/trained model
# models can be downloaded here: https://github.com/fizyr/keras-retinanet/releases
model_path = r"C:\Users\Josh Williams\Retinanet\TrainingNew\snapshots\model_infrence.h5"

# load retinanet model
model = models.load_model(model_path, backbone_name='resnet50')

# convert to infer if needed
# model = models.convert_model(model)

# print(model.summary())

# load label to names mapping for visualization purposes
labels_to_names = {1: 'Cirque'}

clear_output()

## Run detection on example

In [17]:
test_image_list = glob.glob(r"C:\Users\Josh Williams\Retinanet\TrainingNew\Tiles\output1024\*.png")
threshold_list = [0.2]

In [24]:
test_image_list

['C:\\Users\\Josh Williams\\Retinanet\\TrainingNew\\Tiles\\output1024\\tile_0-0.png',
 'C:\\Users\\Josh Williams\\Retinanet\\TrainingNew\\Tiles\\output1024\\tile_0-1024.png',
 'C:\\Users\\Josh Williams\\Retinanet\\TrainingNew\\Tiles\\output1024\\tile_0-1280.png',
 'C:\\Users\\Josh Williams\\Retinanet\\TrainingNew\\Tiles\\output1024\\tile_0-1536.png',
 'C:\\Users\\Josh Williams\\Retinanet\\TrainingNew\\Tiles\\output1024\\tile_0-1792.png',
 'C:\\Users\\Josh Williams\\Retinanet\\TrainingNew\\Tiles\\output1024\\tile_0-2048.png',
 'C:\\Users\\Josh Williams\\Retinanet\\TrainingNew\\Tiles\\output1024\\tile_0-2304.png',
 'C:\\Users\\Josh Williams\\Retinanet\\TrainingNew\\Tiles\\output1024\\tile_0-256.png',
 'C:\\Users\\Josh Williams\\Retinanet\\TrainingNew\\Tiles\\output1024\\tile_0-2560.png',
 'C:\\Users\\Josh Williams\\Retinanet\\TrainingNew\\Tiles\\output1024\\tile_0-2816.png',
 'C:\\Users\\Josh Williams\\Retinanet\\TrainingNew\\Tiles\\output1024\\tile_0-3072.png',
 'C:\\Users\\Josh William

In [35]:
boxes_,scores_,labels_,files_ = [] , [] ,[] ,[]
dfResults = pd.DataFrame()

# loop through test images
for test_image in test_image_list[:10]:

    # loop through thresholds
    for threshold in threshold_list:
    
#         # load image
        image = read_image_bgr(test_image)
#         print(image.shape)
        if image.shape[0]* image.shape[1]== 256*256:
#             sasfdsaf
            image = preprocess_image(image)
            image, scale = resize_image(image)
    
            start = time.time()
            boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
            boxes2 =boxes /scale
            
            for b,s,l in zip(boxes[0],scores[0],labels[0]):
                boxes_.append(",".join(list(map(str,list(map(int,b))))))
                scores_.append(s)
                labels_.append(l)
                files_.append(test_image.replace(".png",".tif"))
    
    
dfResults["files"]= files_
dfResults["scores"]= scores_
dfResults["labels"]= labels_
dfResults["boxes"] = boxes_
   

(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)


In [38]:
image.shape

(800, 800, 3)

In [27]:
print(test_image)

C:\Users\Josh Williams\Retinanet\TrainingNew\Tiles\output1024\tile_768-768.png


In [19]:
dfResults[dfResults.scores>0.2].to_csv("forPolygonsN5FinalN6Mars1024.csv",index=False)

In [21]:
dfResults[dfResults.scores>0.2]

,files,scores,labels,boxes
0,C:\Users\Josh Williams\Retinanet\TrainingNew\T...,0.636038,1,"128,0,456,303"
1,C:\Users\Josh Williams\Retinanet\TrainingNew\T...,0.517213,1,"326,191,555,398"
2,C:\Users\Josh Williams\Retinanet\TrainingNew\T...,0.367622,1,"20,0,355,273"
3,C:\Users\Josh Williams\Retinanet\TrainingNew\T...,0.351130,1,"297,125,609,441"
4,C:\Users\Josh Williams\Retinanet\TrainingNew\T...,0.323438,1,"383,257,532,399"
...,...,...,...,...
201314,C:\Users\Josh Williams\Retinanet\TrainingNew\T...,0.206533,1,"764,580,799,638"
201315,C:\Users\Josh Williams\Retinanet\TrainingNew\T...,0.205440,1,"253,185,592,517"
201316,C:\Users\Josh Williams\Retinanet\TrainingNew\T...,0.204318,1,"687,739,799,799"
201317,C:\Users\Josh Williams\Retinanet\TrainingNew\T...,0.202875,1,"380,182,716,518"
